In [1]:
import pathlib

from cfdmod.use_cases.snapshot.camera import take_snapshot
from cfdmod.use_cases.snapshot.config import SnapshotConfig
from cfdmod.use_cases.snapshot.process_image import process_image

project_path = pathlib.Path("/home/ubuntu/Documentos/Repositories/insight/Docker/local/volume/PrologisCajamar4_final")

cfg_path = project_path / "G100" / "configs" / "snapshot_params.yaml"
cfg = SnapshotConfig.from_file(cfg_path)

output_path = project_path / "G100" / "images"
vtp_path = project_path / "G100" / "env_max.vtp"

# for image_cfg in [img for img in cfg.images if img.scalar_label == "cp_xtr_min"]:
for image_cfg in [img for img in cfg.images if img.scalar_label == "cp_xtr_max"]:
    take_snapshot(
        scalar_name=image_cfg.scalar_label,
        file_path=vtp_path,
        output_path=output_path / f"{image_cfg.image_label}.png",
        colormap_params=cfg.colormap,
        projection_params=cfg.projection,
        camera_params=cfg.camera,
    )
    process_image(
        image_path=output_path / f"{image_cfg.image_label}.png",
        output_path=output_path / f"{image_cfg.image_label}.png",
        crop_cfg=cfg.camera.crop,
    )

TypeError: SetFocalPoint argument 1: expected a sequence of 3 values, got 6 values